In [1]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import scipy.io as scio
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import torchvision
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch
import numpy as np

import yaml
import os
import math
import time
import yaml
import uuid
import pickle
import logging
import argparse
from pathlib import Path

import numpy as np
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl

from sklearn.mixture import GaussianMixture
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score
from ast import literal_eval as make_tuple

import utils
from dataset import DataGenerator, ContrastiveDataGenerator
from projector_plugin import ProjectorPlugin

import torchvision
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch

import matplotlib.pyplot as plt

/Users/gokberk/Desktop/eth_courses/constrastive-DC-GMM/semester_project/lib/python3.8/site-packages/sklearn/utils/linear_assignment_.py:18: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  warnings.warn(


In [ ]:
path = '../dataset/stl10/stl10_matlab/'
data=scio.loadmat(path+'train.mat')
train_X = data['X']
train_Y = data['y'].squeeze()
data=scio.loadmat(path+'test.mat')
test_X = data['X']
test_Y = data['y'].squeeze()
X = []
Y = []
X.append(train_X)
X.append(test_X)
Y.append(train_Y)
Y.append(test_Y)
X = np.concatenate(X,axis=0)
Y = np.concatenate(Y,axis=0)
X = np.reshape(X,(-1,3,96,96))
X = np.transpose(X,(0,1,3,2))

# preprocessing
image_train = X.astype('float32')/255
image_train[:,0,:,:] = (image_train[:,0,:,:] - 0.485)/0.229
image_train[:,1,:,:] = (image_train[:,1,:,:] - 0.456)/0.224
image_train[:,2,:,:] = (image_train[:,2,:,:] - 0.406)/0.225
image_train = torch.from_numpy(image_train)

In [ ]:
res50_model = torchvision.models.resnet50(pretrained=True)
res50_conv = nn.Sequential(*list(res50_model.children())[:-2])
res50_conv.eval()

In [ ]:
total_output = res50_conv(torch.tensor(image_train[:2]))
result = torch.sum(torch.sum(total_output,dim=-1),dim=-1)/9

In [ ]:
result

In [ ]:
path = '../dataset/stl10/stl10_matlab/'
data=scio.loadmat(path+'train.mat')
train_X = data['X']
train_Y = data['y'].squeeze()
data=scio.loadmat(path+'test.mat')
test_X = data['X']
test_Y = data['y'].squeeze()
X = []
Y = []
X.append(train_X)
X.append(test_X)
Y.append(train_Y)
Y.append(test_Y)
X = np.concatenate(X,axis=0)
Y = np.concatenate(Y,axis=0)
X = np.reshape(X,(-1,3,96,96))
X = np.transpose(X,(0,1,3,2))
X = np.transpose(X,(0,2,3,1))

In [ ]:
inputs = tf.keras.Input(X.shape[1:])
x = tf.keras.applications.resnet_v2.preprocess_input(inputs)
outputs = tf.keras.applications.ResNet50V2(include_top=False, weights='imagenet', pooling='avg')(x)
feature_extractor = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
feature_extractor(X[:2], training=False)

In [ ]:
X.dtype

In [ ]:
with open('../config.yml') as file:

    cfg = yaml.load(file, Loader=yaml.FullLoader)

    print(cfg)

if cfg['experiment']['name'] is None:
    timestr = time.strftime("%Y%m%d-%H%M%S")
    ex_name = "{}_{}".format(str(timestr), uuid.uuid4().hex[:5])
else:
    ex_name = cfg['experiment']['name']

experiment_path = os.path.join(cfg['dir']['logging'], cfg['dataset']['name'], ex_name)
Path(experiment_path).mkdir(parents=True)

performance_logger = utils.get_logger(experiment_path, ex_name)

performance_logger.info(f'cfg: {cfg}')
performance_logger.info(f'num GPUs: {len(tf.config.list_physical_devices("GPU"))}')

alpha = utils.get_alpha(cfg)

x_train, x_test, y_train, y_test = utils.get_data(cfg)
inp_shape = x_train.shape[1:]

feature_extractor = utils.get_feature_extractor(inp_shape)

generator = DataGenerator(x_train, y_train, num_constrains=cfg['training']['num_constrains'], alpha=alpha, q=cfg['training']['q'],
                    batch_size=cfg['training']['batch_size'], ml=cfg['training']['ml'], feature_extractor=feature_extractor)
train_generator = generator.gen()

In [ ]:
b = next(train_generator)
b[0][1]

In [ ]:
z = b[1]['output_4']

In [ ]:
tf.concat((z,z),axis=0)

In [ ]:
def get_transformation_pipeline():
    
    transformation = tf.keras.Sequential([   
            tf.keras.layers.experimental.preprocessing.RandomCrop(height=48, width=72),
            tf.keras.layers.experimental.preprocessing.Resizing(height=96, width=96),
            tf.keras.layers.experimental.preprocessing.RandomFlip(mode='horizontal'),
            tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1)
        ])

    return transformation

In [ ]:
transformations_1 = get_transformation_pipeline()

transformations_2 = get_transformation_pipeline()

indexes = [10, 50, 57, 202]
x_batch = tf.gather(x_train,indexes)
x_batch_transformed_1 = transformations_1(x_batch, training=True)
x_batch_transformed_2 = transformations_2(x_batch, training=True)

In [ ]:
%matplotlib inline
i = 0
plt.figure()
f, axarr = plt.subplots(1,2) 
axarr[0].imshow(x_batch_transformed_1[i].numpy().astype(np.uint8))
axarr[1].imshow(x_batch_transformed_2[i].numpy().astype(np.uint8))

In [ ]:
import itertools
batch_size = 4
alpha = 1000


all_pairs = list(itertools.product(range(batch_size), range(batch_size)))

ind1 = []
ind2 = []
data = []

def create_pair_relation(pair):
    el1 = pair[0]
    el2 = pair[1]

    if el1 == el2:
        ind1.append(el1)
        ind2.append(el2+batch_size)
        data.append(1)

        ind1.append(el2+batch_size)
        ind2.append(el1)
        data.append(1)
    
    else:
        ind1.append(el1)
        ind2.append(el2)
        data.append(-1)

        ind1.append(el1)
        ind2.append(el2+batch_size)
        data.append(-1)

        ind1.append(el2+batch_size)
        ind2.append(el1)
        data.append(-1)

        # do not forget to add
        # el1+batch_size, el2+batch_size

    return

_ = list(map(create_pair_relation,all_pairs))

ind1 = np.array(ind1, dtype=np.int32)
ind2 = np.array(ind2, dtype=np.int32)
data = np.array(data) * 10000.

In [ ]:
ind1

In [2]:
with open('../config.yml') as file:

    cfg = yaml.load(file, Loader=yaml.FullLoader)

    print(cfg)

alpha = utils.get_alpha(cfg)

x_train, x_test, y_train, y_test = utils.get_data(cfg)
inp_shape = x_train.shape[1:]

feature_extractor = utils.get_feature_extractor(inp_shape)

generator = ContrastiveDataGenerator(x_train, y_train, alpha=alpha,
                    batch_size=cfg['training']['batch_size'], feature_extractor=feature_extractor)

ds = generator.gen()

{'experiment': {'name': None, 'runs': 1, 'pretrain': False, 'epochs_pretrain': 10, 'lr_pretrain': 0.001, 'save_model': False, 'save_embedding': True}, 'dataset': {'name': 'STL10'}, 'dir': {'data': None, 'checkpoint': '../checkpoints', 'logging': '../logs', 'pretrain': '../pretrain'}, 'training': {'epochs': 1, 'num_constrains': 6000, 'batch_size': 16, 'alpha': 10000, 'q': 0, 'learning_rate': 0.001, 'beta_1': 0.9, 'beta_2': 0.999, 'ml': 0, 'decay_rate': 0.9, 'epochs_lr': 20, 'lrs': True}, 'model': {'latent_dim': 10, 'num_clusters': 10, 'activation': None, 'type': 'FC'}}


In [12]:
b = next(ds)

In [14]:
b

((<tf.Tensor: shape=(32, 2048), dtype=float32, numpy=
  array([[0.        , 2.509237  , 0.253045  , ..., 1.0514897 , 2.3187969 ,
          1.2039105 ],
         [0.13968697, 0.        , 0.        , ..., 0.26970464, 0.        ,
          0.19869316],
         [0.43126166, 0.        , 1.9545751 , ..., 0.79726005, 0.1108368 ,
          0.        ],
         ...,
         [0.07388941, 0.        , 0.46614593, ..., 0.25182188, 0.        ,
          0.6630578 ],
         [0.04083608, 0.45978117, 0.4740952 , ..., 0.69016105, 0.        ,
          0.03596606],
         [0.04201619, 0.        , 0.06456596, ..., 0.        , 3.227163  ,
          0.34051818]], dtype=float32)>,
  (array([ 0, 16,  0,  0, 17,  0,  0, 18,  0,  0, 19,  0,  0, 20,  0,  0, 21,
           0,  0, 22,  0,  0, 23,  0,  0, 24,  0,  0, 25,  0,  0, 26,  0,  0,
          27,  0,  0, 28,  0,  0, 29,  0,  0, 30,  0,  0, 31,  1,  1, 16,  1,
          17,  1,  1, 18,  1,  1, 19,  1,  1, 20,  1,  1, 21,  1,  1, 22,  1,
           1, 